In [ ]:
#!/usr/bin/env/ python
"""
sarsa_learner.py
An easy-to-follow script to train, test and evaluate a SARSA agent on the Mountain Car
problem using the OpenAI Gym. | Ronak Mehta
"""
import gym
import numpy as np

# Assign the hyperparameters
MAX_NUM_EPISODES = 6000 # Please increase the MAX_NUM_EPISODES until it reaches best-reward value of -119.0
STEPS_PER_EPISODE = 200 #  This is specific to MountainCar. May change with env
EPSILON_MIN = 0.005
max_num_steps = MAX_NUM_EPISODES * STEPS_PER_EPISODE
EPSILON_DECAY = 500 * EPSILON_MIN / max_num_steps
ALPHA = 0.05  # Learning rate
GAMMA = 0.98  # Discount factor
NUM_DISCRETE_BINS = 30  # Number of bins to Discretize each observation dim


class SARSA(object):
    def __init__(self, env):
        self.obs_shape = env.observation_space.shape
        self.obs_high = env.observation_space.high
        self.obs_low = env.observation_space.low
        self.obs_bins = NUM_DISCRETE_BINS  # Number of bins to Discretize each observation dim
        self.bin_width = (self.obs_high - self.obs_low) / self.obs_bins
        self.action_shape = env.action_space.n
        # Create a multi-dimensional array (aka. Table) to represent the
        # Q-values
        self.Q = np.zeros((self.obs_bins + 1, self.obs_bins + 1,
                           self.action_shape))  # (51 x 51 x 3)
        self.alpha = ALPHA  # Learning rate
        self.gamma = GAMMA  # Discount factor
        self.epsilon = 1.0

    def discretize(self, obs):
        return tuple(((obs - self.obs_low) / self.bin_width).astype(int))

    def get_action(self, obs):
        # Epsilon-Greedy action selection
        if self.epsilon > EPSILON_MIN:
            self.epsilon -= EPSILON_DECAY
        if np.random.random() > self.epsilon:
            discretized_obs = self.discretize(obs)
            return np.argmax(self.Q[discretized_obs])
        else:  # Choose a random action
            return np.random.choice([a for a in range(self.action_shape)])

    def learn(self, obs, action, reward, next_obs):
        discretized_obs = self.discretize(obs)
        discretized_next_obs = self.discretize(next_obs)
        # Action for the next state using epsilon greedy
        next_action = agent.get_action(next_obs)
        td_target = reward + self.gamma * self.Q[discretized_next_obs][next_action]
        td_error = td_target - self.Q[discretized_obs][action]
        self.Q[discretized_obs][action] += self.alpha * td_error
        return next_action

def train(agent, env):
    best_reward = -float('inf')
    for episode in range(MAX_NUM_EPISODES):
        done = False
        obs = env.reset()
        total_reward = 0.0
        action = agent.get_action(obs)
        while not done:
            next_obs, reward, done, info = env.step(action)
            # Get the next_action and assign it to action
            next_action = agent.learn(obs, action, reward, next_obs)
            action = next_action
            obs = next_obs
            total_reward += reward
        if total_reward > best_reward:
            best_reward = total_reward
        print("Episode#:{} reward:{} best_reward:{} eps:{}".format(episode,
                                     total_reward, best_reward, agent.epsilon))
    # Return the trained policy
    return np.argmax(agent.Q, axis=2)


def test(agent, env, policy):
    done = False
    obs = env.reset()
    total_reward = 0.0
    while not done:
        action = policy[agent.discretize(obs)]
        next_obs, reward, done, info = env.step(action)
        obs = next_obs
        total_reward += reward
    return total_reward


if __name__ == "__main__":
    env = gym.make('MountainCar-v0')
    agent = SARSA(env)
    learned_policy = train(agent, env)
    # Use the Gym Monitor wrapper to evalaute the agent and record video
    gym_monitor_path = "./gym_monitor_output"
    env = gym.wrappers.Monitor(env, gym_monitor_path, force=True)
    for _ in range(1000):
        test(agent, env, learned_policy)
    env.close()

Episode#:0 reward:-200.0 best_reward:-200.0 eps:0.9995812500000084
Episode#:1 reward:-200.0 best_reward:-200.0 eps:0.9991625000000168
Episode#:2 reward:-200.0 best_reward:-200.0 eps:0.9987437500000252
Episode#:3 reward:-200.0 best_reward:-200.0 eps:0.9983250000000337
Episode#:4 reward:-200.0 best_reward:-200.0 eps:0.9979062500000421
Episode#:5 reward:-200.0 best_reward:-200.0 eps:0.9974875000000505
Episode#:6 reward:-200.0 best_reward:-200.0 eps:0.9970687500000589
Episode#:7 reward:-200.0 best_reward:-200.0 eps:0.9966500000000673
Episode#:8 reward:-200.0 best_reward:-200.0 eps:0.9962312500000757
Episode#:9 reward:-200.0 best_reward:-200.0 eps:0.9958125000000841
Episode#:10 reward:-200.0 best_reward:-200.0 eps:0.9953937500000926
Episode#:11 reward:-200.0 best_reward:-200.0 eps:0.994975000000101
Episode#:12 reward:-200.0 best_reward:-200.0 eps:0.9945562500001094
Episode#:13 reward:-200.0 best_reward:-200.0 eps:0.9941375000001178
Episode#:14 reward:-200.0 best_reward:-200.0 eps:0.99371875

Episode#:122 reward:-200.0 best_reward:-200.0 eps:0.948493750001035
Episode#:123 reward:-200.0 best_reward:-200.0 eps:0.9480750000010434
Episode#:124 reward:-200.0 best_reward:-200.0 eps:0.9476562500010518
Episode#:125 reward:-200.0 best_reward:-200.0 eps:0.9472375000010602
Episode#:126 reward:-200.0 best_reward:-200.0 eps:0.9468187500010686
Episode#:127 reward:-200.0 best_reward:-200.0 eps:0.946400000001077
Episode#:128 reward:-200.0 best_reward:-200.0 eps:0.9459812500010855
Episode#:129 reward:-200.0 best_reward:-200.0 eps:0.9455625000010939
Episode#:130 reward:-200.0 best_reward:-200.0 eps:0.9451437500011023
Episode#:131 reward:-200.0 best_reward:-200.0 eps:0.9447250000011107
Episode#:132 reward:-200.0 best_reward:-200.0 eps:0.9443062500011191
Episode#:133 reward:-200.0 best_reward:-200.0 eps:0.9438875000011275
Episode#:134 reward:-200.0 best_reward:-200.0 eps:0.943468750001136
Episode#:135 reward:-200.0 best_reward:-200.0 eps:0.9430500000011444
Episode#:136 reward:-200.0 best_rewar

Episode#:245 reward:-200.0 best_reward:-200.0 eps:0.89698750000207
Episode#:246 reward:-200.0 best_reward:-200.0 eps:0.8965687500020784
Episode#:247 reward:-200.0 best_reward:-200.0 eps:0.8961500000020868
Episode#:248 reward:-200.0 best_reward:-200.0 eps:0.8957312500020952
Episode#:249 reward:-200.0 best_reward:-200.0 eps:0.8953125000021036
Episode#:250 reward:-200.0 best_reward:-200.0 eps:0.894893750002112
Episode#:251 reward:-200.0 best_reward:-200.0 eps:0.8944750000021204
Episode#:252 reward:-200.0 best_reward:-200.0 eps:0.8940562500021288
Episode#:253 reward:-200.0 best_reward:-200.0 eps:0.8936375000021373
Episode#:254 reward:-200.0 best_reward:-200.0 eps:0.8932187500021457
Episode#:255 reward:-200.0 best_reward:-200.0 eps:0.8928000000021541
Episode#:256 reward:-200.0 best_reward:-200.0 eps:0.8923812500021625
Episode#:257 reward:-200.0 best_reward:-200.0 eps:0.8919625000021709
Episode#:258 reward:-200.0 best_reward:-200.0 eps:0.8915437500021793
Episode#:259 reward:-200.0 best_rewar

Episode#:365 reward:-200.0 best_reward:-200.0 eps:0.8467375000030797
Episode#:366 reward:-200.0 best_reward:-200.0 eps:0.8463187500030881
Episode#:367 reward:-200.0 best_reward:-200.0 eps:0.8459000000030965
Episode#:368 reward:-200.0 best_reward:-200.0 eps:0.8454812500031049
Episode#:369 reward:-200.0 best_reward:-200.0 eps:0.8450625000031133
Episode#:370 reward:-200.0 best_reward:-200.0 eps:0.8446437500031218
Episode#:371 reward:-200.0 best_reward:-200.0 eps:0.8442250000031302
Episode#:372 reward:-200.0 best_reward:-200.0 eps:0.8438062500031386
Episode#:373 reward:-200.0 best_reward:-200.0 eps:0.843387500003147
Episode#:374 reward:-200.0 best_reward:-200.0 eps:0.8429687500031554
Episode#:375 reward:-200.0 best_reward:-200.0 eps:0.8425500000031638
Episode#:376 reward:-200.0 best_reward:-200.0 eps:0.8421312500031722
Episode#:377 reward:-200.0 best_reward:-200.0 eps:0.8417125000031807
Episode#:378 reward:-200.0 best_reward:-200.0 eps:0.8412937500031891
Episode#:379 reward:-200.0 best_rew

Episode#:485 reward:-200.0 best_reward:-200.0 eps:0.7964875000040894
Episode#:486 reward:-200.0 best_reward:-200.0 eps:0.7960687500040978
Episode#:487 reward:-200.0 best_reward:-200.0 eps:0.7956500000041062
Episode#:488 reward:-200.0 best_reward:-200.0 eps:0.7952312500041147
Episode#:489 reward:-200.0 best_reward:-200.0 eps:0.7948125000041231
Episode#:490 reward:-200.0 best_reward:-200.0 eps:0.7943937500041315
Episode#:491 reward:-200.0 best_reward:-200.0 eps:0.7939750000041399
Episode#:492 reward:-200.0 best_reward:-200.0 eps:0.7935562500041483
Episode#:493 reward:-200.0 best_reward:-200.0 eps:0.7931375000041567
Episode#:494 reward:-200.0 best_reward:-200.0 eps:0.7927187500041651
Episode#:495 reward:-200.0 best_reward:-200.0 eps:0.7923000000041736
Episode#:496 reward:-200.0 best_reward:-200.0 eps:0.791881250004182
Episode#:497 reward:-200.0 best_reward:-200.0 eps:0.7914625000041904
Episode#:498 reward:-200.0 best_reward:-200.0 eps:0.7910437500041988
Episode#:499 reward:-200.0 best_rew

...

Episode#:5496 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5497 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5498 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5499 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5500 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5501 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5502 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5503 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5504 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5505 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5506 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5507 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5508 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5509 reward:-200.0 best_reward:-117.0 eps:0.00499791667

Episode#:5626 reward:-195.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5627 reward:-167.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5628 reward:-194.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5629 reward:-162.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5630 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5631 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5632 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5633 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5634 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5635 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5636 reward:-170.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5637 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5638 reward:-188.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5639 reward:-200.0 best_reward:-117.0 eps:0.00499791667

Episode#:5742 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5743 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5744 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5745 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5746 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5747 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5748 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5749 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5750 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5751 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5752 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5753 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5754 reward:-178.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5755 reward:-200.0 best_reward:-117.0 eps:0.00499791667

Episode#:5860 reward:-162.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5861 reward:-198.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5862 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5863 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5864 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5865 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5866 reward:-157.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5867 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5868 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5869 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5870 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5871 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5872 reward:-163.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5873 reward:-200.0 best_reward:-117.0 eps:0.00499791667

Episode#:5974 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5975 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5976 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5977 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5978 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5979 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5980 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5981 reward:-198.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5982 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5983 reward:-195.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5984 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5985 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5986 reward:-200.0 best_reward:-117.0 eps:0.004997916674270256
Episode#:5987 reward:-200.0 best_reward:-117.0 eps:0.00499791667